# Example 4: Pipelines

## Set Up

In [1]:
import os
os.chdir("..")
import autompc as ampc
import numpy as np

def doubleint_dynamics(y, u):
    x, dx = y
    return np.array([dx, u])

def dt_doubleint_dynamics(y,u,dt):
    y += dt * doubleint_dynamics(y,u[0])
    return y

def uniform_random_generate(system, task, dynamics, rng, init_min, init_max, 
        traj_len, n_trajs):
    trajs = []
    for _ in range(n_trajs):
        state0 = [rng.uniform(minval, maxval, 1)[0] for minval, maxval 
                in zip(init_min, init_max)]
        y = state0[:]
        traj = ampc.zeros(system, traj_len)
        traj.obs[:] = y
        umin, umax = task.get_ctrl_bounds().T
        for i in range(traj_len):
            traj[i].obs[:] = y
            u = rng.uniform(umin, umax, 1)
            y = dynamics(y, u)
            traj[i].ctrl[:] = u
        trajs.append(traj)
    return trajs

simple_sys = ampc.System(["x", "y"], ["u"])
simple_sys.dt = 0.05

/usr/lib/python3/dist-packages/pyparsing.py:1745: FutureWarning: Possible set intersection at position 3
  self.re = re.compile( self.reString )


Next, we will define the task

In [4]:
from autompc.tasks import Task
from autompc.costs import QuadCost

Q = np.eye(2)
R = np.eye(1)
F = np.eye(2)
cost = QuadCost(simple_sys, Q, R, F, goal=[-1,0])
task = Task(simple_sys)
task.set_cost(cost)
task.set_ctrl_bound("u", -20.0, 20.0)

Now, we generate the training trajectories

In [9]:
rng = np.random.default_rng(42)
dynamics = lambda x, u: dt_doubleint_dynamics(x, u, dt=0.05)
trajs = uniform_random_generate(simple_sys, task, 
                dynamics, rng, init_min=-np.ones(2), 
                init_max=np.ones(2), traj_len=100, n_trajs=100)

## Factories
AutoMPC pipelines use Factories to generate models, costs, and controllers from their configurations.

Here is an example for the SINDy model:

In [10]:
from autompc.sysid import SINDyFactory

model_factory = SINDyFactory(simple_sys)
model_factory.get_configuration_space()

Configuration space object:
  Hyperparameters:
    lasso_alpha_log10, Type: UniformFloat, Range: [-5.0, 2.0], Default: 0.0
    method, Type: Categorical, Choices: {lstsq, lasso}, Default: lstsq
    poly_basis, Type: Categorical, Choices: {true, false}, Default: false
    poly_degree, Type: UniformInteger, Range: [2, 8], Default: 3
    trig_basis, Type: Categorical, Choices: {true, false}, Default: false
    trig_freq, Type: UniformInteger, Range: [1, 8], Default: 1
  Conditions:
    lasso_alpha_log10 | method in {'lasso'}
    poly_degree | poly_basis in {'true'}
    trig_freq | trig_basis in {'true'}

We can use the factory with a given configuration to instantiate (and train the model).

In [13]:
model_cfg = model_factory.get_configuration_space().get_default_configuration()
model_cfg["trig_basis"] = "true"

model = model_factory(model_cfg, trajs)

This works in a similar way for costs and controllers.

In [15]:
from autompc.costs import QuadCostFactory
cost_factory = QuadCostFactory(simple_sys)
cost_cfg = cost_factory.get_configuration_space().get_default_configuration()
cost = cost_factory(cost_cfg, task, trajs)

In [18]:
from autompc.control import IterativeLQRFactory
controller_factory = IterativeLQRFactory(simple_sys)
controller_cfg = controller_factory.get_configuration_space().get_default_configuration()
controller = controller_factory(controller_cfg, task, trajs)

## Pipelines
We can use pipelines to generate the combined configuration space for a model, cost, and controller.

In [19]:
from autompc.pipeline import Pipeline
pipeline = Pipeline(simple_sys, model_factory, cost_factory, controller_factory)
pipeline.get_configuration_space()

Configuration space object:
  Hyperparameters:
    _cost:u_R, Type: UniformFloat, Range: [0.001, 10000.0], Default: 1.0, on log-scale
    _cost:x_F, Type: UniformFloat, Range: [0.001, 10000.0], Default: 1.0, on log-scale
    _cost:x_Q, Type: UniformFloat, Range: [0.001, 10000.0], Default: 1.0, on log-scale
    _cost:y_F, Type: UniformFloat, Range: [0.001, 10000.0], Default: 1.0, on log-scale
    _cost:y_Q, Type: UniformFloat, Range: [0.001, 10000.0], Default: 1.0, on log-scale
    _ctrlr:horizon, Type: UniformInteger, Range: [5, 25], Default: 20
    _model:lasso_alpha_log10, Type: UniformFloat, Range: [-5.0, 2.0], Default: 0.0
    _model:method, Type: Categorical, Choices: {lstsq, lasso}, Default: lstsq
    _model:poly_basis, Type: Categorical, Choices: {true, false}, Default: false
    _model:poly_degree, Type: UniformInteger, Range: [2, 8], Default: 3
    _model:trig_basis, Type: Categorical, Choices: {true, false}, Default: false
    _model:trig_freq, Type: UniformInteger, Range: [1

We can then use the pipeline to instantiate all components

In [22]:
pipeline_cfg = pipeline.get_configuration_space().get_default_configuration()
controller, cost, model = pipeline(pipeline_cfg, task, trajs)

It is also easy for us to fix a particular component of the pipeline.  Here is an example of pipeline with a pre-trained model

In [ ]:
pipeline_pretrain_model = Pipeline(sim)